In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial

from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *


In [2]:
wos = Query.objects.get(pk=365)
scopus = Query.objects.get(pk=354)

In [ ]:
alldocs = Doc.objects.filter(query=wos) | Doc.objects.filter(query=scopus)
alldocs.count()
print(Doc.objects.filter(query=scopus).count())

254395


In [ ]:
Doc.objects.filter(query=scopus).update(scopus=True)
bypy = alldocs.values('PY').annotate(
    only_wos = models.Sum(
        models.Case(
            models.When(wos=True,scopus=False,then=1),default=0, output_field=models.IntegerField()
        )
    )
)
bypy2 = alldocs.values('PY').annotate(
    scopus = models.Sum(
        models.Case(
            models.When(scopus=True,wos=False,then=1),default=0, output_field=models.IntegerField()
        )
    )
)
bypy3 = alldocs.values('PY').annotate(
    both = models.Sum(
        models.Case(
            models.When(scopus=True,wos=True,then=1),default=0, output_field=models.IntegerField()
        )
    )
)



In [ ]:
bypy = pd.DataFrame.from_dict(list(bypy))
bypy = bypy.merge(pd.DataFrame.from_dict(list(bypy2)))
bypy = bypy.merge(pd.DataFrame.from_dict(list(bypy3)))


bypy = bypy[bypy['PY']>1980]#.reset_index(drop=True)
bypy = bypy.sort_values(by='PY', ascending=1).reset_index(drop=True)

In [ ]:


bypy['AP'] = pd.cut(bypy.PY,bins=[0,1985,1990.1,1995.1,2001.1,2007.1,2013.1,9999],labels=["NA","AR1","AR2","AR3","AR4","AR5","AR6"])
bypy.head(5)

In [ ]:
plt.rcParams["figure.figsize"] = [12,8]

bypy=bypy[bypy['PY']<2017]

edge="grey"
color = "white"

colors = {'NA':"white",'AR1':"#D53E4F", 'AR2':'#FC8D59', 'AR3':'#FEE08B', 'AR4':'#E6F598','AR5':'#99D594','AR6':'#3288BD'}

p1 = plt.bar(bypy.PY,bypy['only_wos'],color=bypy['AP'].apply(lambda x: colors[x]),edgecolor=edge, hatch=2*"/")
p2 = plt.bar(bypy.PY,bypy['both'],color=bypy['AP'].apply(lambda x: colors[x]),edgecolor=edge, hatch=2*"x",bottom=bypy['only_wos'])
p3 = plt.bar(bypy.PY,bypy['scopus'],color=bypy['AP'].apply(lambda x: colors[x]),edgecolor=edge, hatch=2*"\\",bottom=bypy['only_wos']+bypy['both'])

plt.legend(
    (p2[6],p2[10],p2[15],p2[22],p2[28],p2[35],p3[0],p2[0],p1[0]),
    ('AR1','AR2','AR3','AR4','AR5','AR6','Only Scopus','Both','Only WoS')
)


plt.savefig('../plots/wos_scopus_docs_time.png',bbox_inches='tight')
plt.show()

In [ ]:
bypy['total'] = bypy.only_wos + bypy.scopus + bypy.both
byap = bypy.groupby(['AP']).sum()
print(byap)